# Imports

In [1]:
#Imports
from modules.preamble import *
import json

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from skfuzzy import cmeans

from modules.PFS import PFS_layer, PFS_CP
from modules.kappa import *
from modules.RNN_utils import *

%matplotlib inline

# Loading the data

In [2]:
#Load the data
df_train = pd.read_hdf(os.path.join(data_base_path, 'modeling_data/df_train.hdf'))
df_val = pd.read_hdf(os.path.join(data_base_path, 'modeling_data/df_val.hdf'))
df_test = pd.read_hdf(os.path.join(data_base_path, 'modeling_data/df_test.hdf'))

X_train = pd.read_hdf(os.path.join(data_base_path, 'modeling_data/X_train.hdf')).values
X_val = pd.read_hdf(os.path.join(data_base_path, 'modeling_data/X_val.hdf')).values
X_test = pd.read_hdf(os.path.join(data_base_path, 'modeling_data/X_test.hdf')).values

y_train = pd.read_hdf(os.path.join(data_base_path, 'modeling_data/y_train.hdf')).values.ravel() #flat
y_val = pd.read_hdf(os.path.join(data_base_path, 'modeling_data/y_val.hdf')).values.ravel()
y_test = pd.read_hdf(os.path.join(data_base_path, 'modeling_data/y_test.hdf')).values.ravel()

instance_weights_train = pd.read_hdf(os.path.join(data_base_path, 'modeling_data/instance_weights_train.hdf')).values.ravel()

## Comment w.r.t. class weight
Must (awkwardly) pass these as aggregated instance + class weights (multiply them) to tensorflow. Default interface offers no option to apply instance + class weights during training and only sample weights during testing (i.e. not possible when passing both arguments "class_weight" and "sample_weight"). Tested this awkward aspect extensively due to weird first results - pretty sure TF does not offer the feature we need out-of-the-box.

In [3]:
# Adjust sample weight in training w.r.t classes (i.e. multiply by class weight).

class_weights = compute_class_weight('balanced', [0,1], y_train) #Class weights as array
class_weights = {0: class_weights[0], 1: class_weights[1]} #Convert to dictionary

instance_class_weights_train = np.array(
    [sample_weight * class_weights[y] for sample_weight, y in zip(instance_weights_train, y_train)]
)

In [4]:
#Quick checking of data shapes to see if load was correct
print(df_train.shape, df_val.shape, df_test.shape)
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)
print(instance_weights_train.shape)

(688457, 31) (117419, 31) (118967, 31)
(688457, 26) (117419, 26) (118967, 26)
(688457,) (117419,) (118967,)
(688457,)


# Setup
It seems that predicting for variable-length sequences with sample weights stretches the functionality of TF to its boundaries - we hit some bugs in the library here. Fitting the GRU thus requires some workarounds:

## Fitting to variable-length time sequences
The model.fit and model.predict functions (and all other related predict functions) do not work with variable-length sequences: it seems that Tensorflow attempts to convert all sequences to Tensor format first, which is impossible due to the varying length (each dimension must be constant in a Tensor) and raises an error. There does not seem any way to disable this easily. 
* For feeding sequences with a batch size of 1 without any zero-padding, we must use a custom sequence generator, that yields a single time series in one batch, including features & labels & (possibly) sample weigths.
    * Note that our implementation also allows for bigger batches than size 0, with possibly zero padding. However, specifying a batch size of 1 results in no zero-padding.
* The .predict function, and all related predict functions, try to convert the output of all batches to a single tensor after predicting. Thus, these functions result in an error when used on variable-length sequences, even when using a custom sequence generator!
    * Workaround 1: Predict for every batch individually using a for-loop. 
        * This is very slow and results in excessive printing of warnings - TF warns you that you should perform vectorized computations and not use Python for-loops.
    * Workaround 2: Predict with all samples (all time series for all patients) in a single batch. To do this, zero-pad every sequence to the longest sequence length, predict, prune the zero padding again, and finally flatten the predictions back to a 1D array.
        * This seems to run extremely fast without warnings and is thus the approach of choice.
    
## Sample weigths + variable-length time sequences
There seems to be a bug when using sample weights for cells that take sequences as input (such as the GRU): setting sample_weight_mode to "temporal" in model.compile, as recommended in the documentation, still only seems to result in the use of one sample weight per patient. 
* Workaround: use one sample weight per patient time sequence. As the sample weight is a constant for all timesteps in a sequence anyway, this should yield the same result (i.e. it corresponds to moving the sample weight constant from *inside* the summation over all timesteps in a sequence in the loss function to a just-once multiplication *outside* summing the loss of all timesteps).

In [5]:
X_seq_train, y_seq_train, instance_class_weights_seq_train = to_seq(df_train,
                                                                    'subject_id',
                                                                     X_train,
                                                                     y_train,
                                                                     instance_class_weights_train)

X_seq_val, y_seq_val = to_seq(df_val,
                              'subject_id',
                               X_val,
                               y_val)                                                                   

In [6]:
#Brief check: inspect some shapes of arrays
print(len(X_seq_train), len(X_seq_val))
print(len(instance_class_weights_seq_train))
print(len(y_seq_train), len(y_seq_val))

1691 362
1691
1691 362


In [7]:
def generate_random_gru(input_shape):
    """
    Generate a random densely connected neural network & return the configuration + model.
    The possible value ranges for the hyperparameters are as specified in the thesis
        - they are hard-coded.
    
    Returns: n_layers, n_units_per_layer, dropouts, model
    """
    #Choose number of layers & number of units per layer
    n_layers = int(np.random.randint(1,3,1)[0])
    n_units_per_layer, l1_reg_per_layer, l2_reg_per_layer = [], [], []
    for i in range(n_layers):
        n_units_per_layer.append(int(np.random.randint(2,21,1)[0]))

    #Setup the DL model
    dropouts = []
    model = tf.keras.Sequential()
    for i, n_units in enumerate(n_units_per_layer):
        if (i==0):
            model.add(tf.keras.layers.GRU(n_units,
                                          input_shape=input_shape,
                                          return_sequences=True))
        else:
            #Add possible dropout and the dense layer with the random number of units
            dropout_rate = np.random.uniform(0, 0.1)
            dropouts.append(dropout_rate)
            model.add(tf.keras.layers.GRU(n_units, return_sequences=True, dropout=dropout_rate))

    model.add(tf.keras.layers.Dense(1, activation='sigmoid')) #Add the output layer, which always has 1 hidden unit and sigmoid output

    model.compile(optimizer='Adam',
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.AUC(curve='ROC', name='ROC_AUC')],
                 )#sample_weight_mode='temporal')
    
    return n_layers, n_units_per_layer, dropouts, model

# Tuning the GRU with all features

NB: Set the results to be stored as raw Python types, otherwise we get issues w/ JSON writing.

In [9]:
#Setup
results = {}
filenr=1 #Alter filenames sometimes such that we have restore points in case the data gets corrupted
iteration = 0

#Use a batch size of 1 to avoid excessive zero-padding and an excessive amount of extra data points to fit the model to
train_gen = Sequence_generator(X_seq_train, y_seq_train, batch_size=1, sample_weight=instance_class_weights_seq_train)
val_gen = Sequence_generator(X_seq_val, y_seq_val, batch_size=1)

In [10]:
#Automated tuning - stop this manually after your time budget has run out.
while(True):
    #Update iteration & file number
    iteration += 1
    if (iteration%5 ==0):
        print(iteration)
        filenr += 1
    
    #store start time of the fit & set random seed
    start_time = time.time()
    np.random.seed(iteration)
    
    #Create a randomized DNN & fit it to the data
    n_layers, n_units_per_layer, dropouts, model = generate_random_gru(input_shape=(None, 26))

    #Instantiate generators with batch size of 1 (to avoid excessive zero-padding due to some long sequences)
    train_gen = Sequence_generator(X_seq_train, y_seq_train, batch_size=1, sample_weight=instance_class_weights_seq_train)
    val_gen = Sequence_generator(X_seq_val, y_seq_val, batch_size=1)
    model.fit(train_gen,
              validation_data=val_gen,
              epochs=100,
              verbose=1,
              callbacks= [tf.keras.callbacks.EarlyStopping(monitor='val_ROC_AUC',
                                                           mode='max',
                                                           patience=1, 
                                                           min_delta=0.05)]) #Stricter early stopping criteria due to more expensive models

    #Compute performance using the workaround
    val_gen = Sequence_generator(X_seq_val, y_seq_val, batch_size=len(X_seq_val)) #Create generator
    y_score_seq = model.predict(val_gen) #Predict for all (padded) sequences
    seq_lengths = [X_seq_val[i].shape[0] for i in range(len(X_seq_val))] #Prune the padding
    y_score = np.vstack([y_score_seq[i][:seq_lengths[i]] for i in range(y_score_seq.shape[0])]) #Flatten the labels again
    roc_auc = roc_auc_score(y_val, y_score)
    auk = auk_score(y_val, y_score)

    #Store the configuration & output (to raw python types, otherwise issues w/ writing JSON)
    results[iteration] = {
        'n_layers': n_layers,
        'n_units_per_layer': n_units_per_layer,
        'dropout_rate_per_layer': dropouts,
        'roc_auc': roc_auc,
        'auk': auk,
        'fitting_time (minutes)': (time.time() - start_time)/60
    }

    # with open(os.path.join(data_base_path, 'model_tuning', 'GRU-all_feats-results-{}.json'.format(filenr)), 'w') as f:
    #     json.dump(results, f, indent=4)

Epoch 1/100
1691/1691 [==============================] - 718s 425ms/step - loss: 0.0061 - ROC_AUC: 0.7417 - val_loss: 0.6499 - val_ROC_AUC: 0.7741
Epoch 2/100
1691/1691 [==============================] - 750s 443ms/step - loss: 0.0048 - ROC_AUC: 0.7544 - val_loss: 0.7639 - val_ROC_AUC: 0.7586
Epoch 1/100
1691/1691 [==============================] - 187s 111ms/step - loss: 0.0057 - ROC_AUC: 0.7134 - val_loss: 0.8136 - val_ROC_AUC: 0.7732
Epoch 2/100
1691/1691 [==============================] - 296s 175ms/step - loss: 0.0045 - ROC_AUC: 0.7620 - val_loss: 0.7359 - val_ROC_AUC: 0.7789
Epoch 1/100
1691/1691 [==============================] - 280s 166ms/step - loss: 0.0078 - ROC_AUC: 0.6584 - val_loss: 0.7899 - val_ROC_AUC: 0.7441
Epoch 2/100
1691/1691 [==============================] - 267s 158ms/step - loss: 0.0055 - ROC_AUC: 0.7333 - val_loss: 0.8644 - val_ROC_AUC: 0.7669
Epoch 1/100
1691/1691 [==============================] - 222s 132ms/step - loss: 0.0057 - ROC_AUC: 0.7302 - val_loss: 

E0525 15:45:49.082839 19588 ultratb.py:152] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\KevinReijnders\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-b1d94ecf6662>", line 28, in <module>
    min_delta=0.05)]) #Stricter early stopping criteria due to more expensive models
  File "C:\Users\KevinReijnders\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py", line 66, in _method_wrapper
    return method(self, *args, **kwargs)
  File "C:\Users\KevinReijnders\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py", line 848, in fit
    tmp_logs = train_function(iterator)
  File "C:\Users\KevinReijnders\Anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 580, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\KevinReijnders\Anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 611, in _call
    return self._stateles

KeyboardInterrupt: 

# Tuning the GRU with the features of the best PFS

NB: Set the results to be stored as raw Python types, otherwise we get issues w/ JSON writing.

In [10]:
#Load the features from the best PFS configuration
with open(os.path.join(data_base_path, 'model_tuning', 'Best-PFS-config.json'), 'r') as f:
    pfs_config = json.load(f)
indices_sel_features = pfs_config['feature_indices']

In [11]:
#Alter sequences such that they only have the features selected in the PFS
X_seq_new = []
for X_seq in X_seq_train:
    X_seq_new.append(
        X_seq[:,indices_sel_features]
    )
X_seq_train = np.array(X_seq_new)

X_seq_new = []
for X_seq in X_seq_val:
    X_seq_new.append(
        X_seq[:,indices_sel_features]
    )
X_seq_val = np.array(X_seq_new)

In [12]:
#Setup
results = {}
filenr=1 #Alter filenames sometimes such that we have restore points in case the data gets corrupted
iteration = 0

#Use a batch size of 1 to avoid excessive zero-padding and an excessive amount of extra data points to fit the model to
train_gen = Sequence_generator(X_seq_train, y_seq_train, batch_size=1, sample_weight=instance_class_weights_seq_train)
val_gen = Sequence_generator(X_seq_val, y_seq_val, batch_size=1)

In [ ]:
#Automated tuning - stop this manually after your time budget has run out.
while(True):
    #Update iteration & file number
    iteration += 1
    if (iteration%5 ==0):
        print(iteration)
        filenr += 1
    
    #store start time of the fit & set random seed
    start_time = time.time()
    np.random.seed(iteration+500) #Different seeds than version with all features
    
    #Create a randomized DNN & fit it to the data
    n_layers, n_units_per_layer, dropouts, model = generate_random_gru(input_shape=(None, len(indices_sel_features)))

    #Instantiate generators with batch size of 1 (to avoid excessive zero-padding due to some long sequences)
    train_gen = Sequence_generator(X_seq_train, y_seq_train, batch_size=1, sample_weight=instance_class_weights_seq_train)
    val_gen = Sequence_generator(X_seq_val, y_seq_val, batch_size=1)
    model.fit(train_gen,
              validation_data=val_gen,
              epochs=100,
              verbose=1,
              callbacks= [tf.keras.callbacks.EarlyStopping(monitor='val_ROC_AUC',
                                                           mode='max',
                                                           patience=1, 
                                                           min_delta=0.05)]) #Stricter early stopping criteria due to more expensive models

    #Compute performance using the workaround
    val_gen = Sequence_generator(X_seq_val, y_seq_val, batch_size=len(X_seq_val)) #Create generator
    y_score_seq = model.predict(val_gen) #Predict for all (padded) sequences
    seq_lengths = [X_seq_val[i].shape[0] for i in range(len(X_seq_val))] #Prune the padding
    y_score = np.vstack([y_score_seq[i][:seq_lengths[i]] for i in range(y_score_seq.shape[0])]) #Flatten the labels again
    roc_auc = roc_auc_score(y_val, y_score)
    auk = auk_score(y_val, y_score)

    #Store the configuration & output (to raw python types, otherwise issues w/ writing JSON)
    results[iteration] = {
        'n_layers': n_layers,
        'n_units_per_layer': n_units_per_layer,
        'dropout_rate_per_layer': dropouts,
        'roc_auc': roc_auc,
        'auk': auk,
        'fitting_time (minutes)': (time.time() - start_time)/60
    }

    # with open(os.path.join(data_base_path, 'model_tuning', 'GRU-pfs_feats-results-{}.json'.format(filenr)), 'w') as f:
    #     json.dump(results, f, indent=4)

Epoch 1/100
1691/1691 [==============================] - 705s 417ms/step - loss: 0.0080 - ROC_AUC: 0.6286 - val_loss: 1.2859 - val_ROC_AUC: 0.6660
Epoch 2/100
1691/1691 [==============================] - 805s 476ms/step - loss: 0.0068 - ROC_AUC: 0.6846 - val_loss: 0.9504 - val_ROC_AUC: 0.6687
Epoch 1/100
1691/1691 [==============================] - 807s 477ms/step - loss: 0.0084 - ROC_AUC: 0.6271 - val_loss: 1.2624 - val_ROC_AUC: 0.7079
Epoch 2/100
1691/1691 [==============================] - 843s 499ms/step - loss: 0.0073 - ROC_AUC: 0.6905 - val_loss: 1.4178 - val_ROC_AUC: 0.7088
Epoch 1/100
1691/1691 [==============================] - 801s 473ms/step - loss: 0.0077 - ROC_AUC: 0.6697 - val_loss: 1.0840 - val_ROC_AUC: 0.6676
Epoch 2/100
1691/1691 [==============================] - 760s 449ms/step - loss: 0.0067 - ROC_AUC: 0.6926 - val_loss: 1.0115 - val_ROC_AUC: 0.6612
Epoch 1/100
  10/1691 [..............................] - ETA: 3:58 - loss: 0.0013 - ROC_AUC: 0.5901

# Quick analysis of the results & storing the best configuration

In [2]:
df_all = pd.read_json(os.path.join(data_base_path, 'model_tuning', 'GRU-all_feats-results-1.json'),
                      orient='index')
df_pfs = pd.read_json(os.path.join(data_base_path, 'model_tuning', 'GRU-pfs_feats-results-1.json'),
                      orient='index')

In [5]:
df_all.sort_values('roc_auc', ascending=False)

,n_layers,n_units_per_layer,dropout_rate_per_layer,roc_auc,auk,fitting_time (minutes)
4,1,[16],[],0.781861,0.141310,10.055467
2,1,[17],[],0.778956,0.139485,8.099269
3,1,[5],[],0.767229,0.133890,9.181591
1,2,"[13, 14]",[0.093255735933865],0.758658,0.129112,24.553569


In [6]:
df_pfs.sort_values('roc_auc', ascending=False)

,n_layers,n_units_per_layer,dropout_rate_per_layer,roc_auc,auk,fitting_time (minutes)
2,2,"[2, 15]",[0.034833486428848],0.709630,0.100522,27.584601
1,2,"[9, 17]",[0.00587862674409],0.668772,0.081022,25.244659
3,2,"[8, 9]",[0.090385537299124],0.661141,0.079320,26.086430


In [7]:
#Export best configs the hyperparameters of the best GRU models
# with open(os.path.join(data_base_path, 'model_tuning', 'Best-GRU-configs.json'), 'w') as f:
#     best_dnn_config = {
#         1: {
#             'features': 'all',
#             'n_layers': 1,
#             'n_units_per_layer': [16],
#             'dropout_rates': [],
#         },
#         2: {
#             'features': 'best_pfs',
#             'n_layers': 2,
#             'n_units_per_layer': [2,15],
#             'dropout_rates': [0.034833486428848],
#         },
#     }
#     json.dump(best_dnn_config, f, indent=4)